In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path_to_your_data = ""

tables_used = ['patient', 'diagnosis', 'treatment', 'medication', 'lab', 'apacheApsVar']
patient_path = path_to_your_data + '/patient.csv'
diagnosis_path = path_to_your_data + '/diagnosis.csv'
treatment_path = path_to_your_data + '/treatment.csv'
medication_path = path_to_your_data + '/medication.csv'
lab_path = path_to_your_data + '/lab.csv'
apache_aps_path = path_to_your_data + '/apacheApsVar.csv'


### Read patient data

In [3]:
patient = pd.read_csv(patient_path)

We can use the 'hospitaldischargestatus' column for survival target variables

In [4]:
# "survival status" -- look at the class imbalance
patient['hospitaldischargestatus'].value_counts()

hospitaldischargestatus
Alive      181104
Expired     18004
Name: count, dtype: int64

In [8]:
# note that we are keeping the patientunitstayid from the first ICU visit
cols_to_keep = ['patientunitstayid', 'uniquepid', 'age', 'gender', 'ethnicity', \
                'los', 'hospitaldischargestatus']
patient_pp = patient[cols_to_keep].dropna()
print(f"After filtering patients NAs, left with {len(patient_pp)} stays")
# filter out patients with age over 89
patient_pp = patient_pp.loc[(patient_pp['age'] != "> 89")]
print(f"After filtering patients with age over 89, left with {len(patient_pp)} stays")
# convert to numeric
patient_pp['age'] = patient_pp['age'].astype(int)
# filter out patients with age less than 18
patient_pp = patient_pp.loc[(patient_pp['age'] >= 18)]
print(f"After filtering patients with age under 18, left with {len(patient_pp)} stays")


After filtering patients NAs, left with 196848 stays
After filtering patients with age over 89, left with 189936 stays
After filtering patients with age under 18, left with 189429 stays


#### SURVIVAL TASK

In [13]:
# categorical variables
categorical_vars = ['gender', 'ethnicity']
continuous_vars = ['age', 'hospitaldischargestatus']
id_vars = ['patientunitstayid', 'uniquepid']
patient_pp_survival = pd.concat([patient_pp[id_vars], \
               patient_pp[continuous_vars], \
               pd.get_dummies(patient_pp[categorical_vars])], axis = 1)
patient_pp_survival['hospitaldischargestatus'] = patient_pp_survival['hospitaldischargestatus'].map(\
                                                                            {"Expired": 1, "Alive": 0})

In [14]:
patient_pp_survival['hospitaldischargestatus'].value_counts()

hospitaldischargestatus
0    165775
1     14766
Name: count, dtype: int64

In [15]:
# mortality task
save_path = ""

mortality_task_files = save_path + 'by_modality/mortality_task'
patient_pp_survival.to_csv(os.path.join(mortality_task_files, "modality1_demographics.csv"))

---

### Determine modalities:
(from paper) "eICU (Pollard et al., 2018) covers 431K visits for 180K
patients admitted to the ICU in the Beth Israel Deaconess Medical Center. We use demographics (age, gender, and ethnicity),,
diagnosis, procedure, medication, lab values, and vital signals as input modalities" "We extract data from the **patient, diagnosis, treatment, medication, lab, and apacheApsVar** tables. Diagnosis, treatment, and medication are recorded as medical codes. We select our cohorts by filtering out visits of patients younger than 18 or older than 89 years old, visits that last longer than 10 days, and visits lasting shorter than 12 hours, as the predictions are made 12 hours after admission."

In [16]:
def string_parsing(df, col_to_parse, thresh=0.05):
    df_nona = df[df[col_to_parse].notna()]
    # for each patient stay id, combine all instances of column and put them in a string with | dividers
    result_df = df_nona.groupby('patientunitstayid').agg({
        col_to_parse : lambda x: "|".join(list(x)),
    }).reset_index()
    print("Grouped by patient unit stay id.")
    # get unique values between | dividers
    result_df[col_to_parse] = result_df[col_to_parse].apply(lambda x: "|".join(list(set(x.split("|")))))
    # get dummies
    dummies = result_df[col_to_parse].str.get_dummies("|")
    print("Derived dummies.")
    # filter dummies with low frequencies
    sums = dummies.sum()
    # total rows
    total = len(dummies)
    # keep drugs present in at least 5% of visits
    threshold= (int)(0.05*total)
    dummies_keep = sums[sums > threshold]
    dummies = dummies[dummies_keep.index]
    print("Filtered dummies.")
    # concat to original ids
    df_pp = pd.concat([result_df['patientunitstayid'], dummies], axis=1)
    print("Preprocessing completing.")
    return df_pp
    

#### Diagnosis table

In [17]:
diagnosis = pd.read_csv(diagnosis_path)
print(f"Shape: {diagnosis.shape}")
diagnosis.head()

Shape: (2710672, 7)


,diagnosisid,patientunitstayid,activeupondischarge,diagnosisoffset,diagnosisstring,icd9code,diagnosispriority
0,4222318,141168,False,72,cardiovascular|chest pain / ASHD|coronary arte...,"414.00, I25.10",Other
1,3370568,141168,True,118,cardiovascular|ventricular disorders|cardiomyo...,NaN,Other
2,4160941,141168,False,72,pulmonary|disorders of the airways|COPD,"491.20, J44.9",Other
3,4103261,141168,True,118,pulmonary|disorders of the airways|COPD,"491.20, J44.9",Other
4,3545241,141168,True,118,cardiovascular|ventricular disorders|congestiv...,"428.0, I50.9",Other


In [18]:
# OPTIONAL: keep entries made within the first 12 hours
# diagnosis = diagnosis[diagnosis['diagnosisoffset'] < (12*60)]

In [19]:
diagnosis_pp = string_parsing(diagnosis, 'diagnosisstring')

Grouped by patient unit stay id.
Derived dummies.
Filtered dummies.
Preprocessing completing.


In [20]:
diagnosis_pp.to_csv(save_path + 'by_modality/mortality_task/modality2_diagnosis.csv')

#### Treatment table

In [21]:
treatment = pd.read_csv(treatment_path)
print(f"Shape: {treatment.shape}")
treatment.head()

Shape: (3688745, 5)


,treatmentid,patientunitstayid,treatmentoffset,treatmentstring,activeupondischarge
0,8399138,242040,198,cardiovascular|hypertension|angiotensin II rec...,False
1,8626134,242040,198,cardiovascular|myocardial ischemia / infarctio...,False
2,8517569,242040,198,infectious diseases|medications|therapeutic an...,False
3,9597686,242040,616,cardiovascular|non-operative procedures|diagno...,False
4,9334096,242040,618,infectious diseases|medications|therapeutic an...,True


In [22]:
# OPTIONAL: keep entries made within the first 12 hours
# treatment = treatment[treatment['treatmentoffset'] < (12*60)]

In [23]:
treatment_pp = string_parsing(treatment, 'treatmentstring')

Grouped by patient unit stay id.
Derived dummies.
Filtered dummies.
Preprocessing completing.


In [24]:
treatment_pp.to_csv(save_path + 'by_modality/mortality_task/modality3_treatment.csv')

#### Medication table

In [25]:
medication = pd.read_csv(medication_path)
print(medication.shape)
medication.head()

/tmp/ipykernel_2526840/1921665291.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  medication = pd.read_csv(medication_path)


(7301853, 15)


,medicationid,patientunitstayid,drugorderoffset,drugstartoffset,drugivadmixture,drugordercancelled,drugname,drughiclseqno,dosage,routeadmin,frequency,loadingdose,prn,drugstopoffset,gtc
0,7426715,141168,309,666,No,No,METOPROLOL TARTRATE 25 MG PO TABS,2102.0,25 3,PO,Q12H SCH,NaN,No,1826,0
1,9643232,141168,1847,1832,No,No,3 ML - IPRATROPIUM-ALBUTEROL 0.5-2.5 (3) MG/...,NaN,3 1,NEBULIZATION,Q4H Resp PRN,NaN,Yes,2047,0
2,10270090,141168,296,1386,No,No,ASPIRIN EC 81 MG PO TBEC,1820.0,81 3,PO,Daily,NaN,No,2390,0
3,9496768,141168,2048,2029,No,No,3 ML - IPRATROPIUM-ALBUTEROL 0.5-2.5 (3) MG/...,NaN,3 1,NEBULIZATION,Q4H Resp PRN,NaN,Yes,2390,0
4,11259680,141168,117,246,No,No,ENOXAPARIN SODIUM 40 MG/0.4ML SC SOLN,NaN,40 3,SC,Daily,NaN,No,1721,0


In [26]:
# OPTIONAL: keep entries made within the first 12 hours
# medication = medication[medication['drugstartoffset'] < (12*60)]

In [27]:
medication_pp = string_parsing(medication, "drugname")

Grouped by patient unit stay id.
Derived dummies.
Filtered dummies.
Preprocessing completing.


In [28]:
medication_pp.to_csv(save_path + 'by_modality/mortality_task/modality4_medication.csv')

#### Lab table

In [29]:
lab = pd.read_csv(lab_path)
print(lab.shape)
lab.head()

(39132531, 10)


,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset
0,52307161,141168,2026,3,fibrinogen,177.0,177,mg/dL,mg/dL,2219
1,50363251,141168,1133,3,PT - INR,2.5,2.5,ratio,NaN,1208
2,49149139,141168,2026,1,magnesium,2.0,2.0,mg/dL,mg/dL,2090
3,50363250,141168,1133,3,PT,26.6,26.6,sec,sec,1208
4,66695374,141168,2141,7,pH,7.2,7.20,NaN,Units,2155


In [30]:
# OPTIONAL: keep entries made within the first 12 hours
# lab = lab[lab['labresultoffset'] < (12*60)]

In [31]:
lab_types = lab['labname'].value_counts()
lab_types

labname
bedside glucose         3175835
potassium               1493261
sodium                  1393205
glucose                 1319496
Hgb                     1298708
                         ...   
HSV 1&2 IgG AB titer         12
NAPA                         10
Procainamide                 10
HIV 1&2 AB                    7
RPR titer                     3
Name: count, Length: 158, dtype: int64

In [32]:
# we keep lab result types with at least 10000 instances in our dataset
# other lab results likely will be too sparse
lab_types_to_keep = lab_types[lab_types > (10000)].index.to_list()
lab_filtered = lab[lab['labname'].isin(lab_types_to_keep)]

In [33]:
# group by patientunitstayid, then pivot on labname column, setting the values as lab result
# for any duplicate lab result type for a stay, we sum the amounts (for ex. there were two glucose tests during 
# a patient stay, then we add the total glucose values) 
lab_pp = lab_filtered.pivot_table(index='patientunitstayid', columns='labname', 
                                    values='labresult', aggfunc='sum')

In [34]:
# all empty lab results we set as 0
lab_pp = lab_pp.fillna(0).reset_index()

In [35]:
lab_pp.to_csv(save_path + 'by_modality/mortality_task/modality5_lab.csv')

#### Apache table (Acute physiology scores)

In [36]:
aps = pd.read_csv(apache_aps_path)
print(aps.shape)
aps.head()

(171177, 26)


,apacheapsvarid,patientunitstayid,intubated,vent,dialysis,eyes,motor,verbal,meds,urine,...,ph,hematocrit,creatinine,albumin,pao2,pco2,bun,glucose,bilirubin,fio2
0,2,141168,0,0,0,4,6,5,0,-1.0,...,-1.00,40.1,2.30,3.1,-1.0,-1.0,27.0,95.0,4.1,-1.0
1,2399659,141178,0,0,0,-1,-1,-1,-1,-1.0,...,-1.00,-1.0,-1.00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,167885,141194,0,0,0,3,6,4,0,-1.0,...,-1.00,27.4,2.51,2.3,-1.0,-1.0,31.0,168.0,0.4,-1.0
3,70691,141197,0,0,0,4,6,5,0,-1.0,...,-1.00,-1.0,-1.00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,2279689,141203,0,1,0,1,3,1,0,-1.0,...,7.45,36.9,0.56,-1.0,51.0,37.0,9.0,145.0,-1.0,100.0


In [37]:
aps_pp = aps.drop(columns=['apacheapsvarid'])

In [38]:
aps_pp.to_csv(save_path + 'by_modality/mortality_task/modality6_aps.csv')

----